In [1]:
import zarr
import xarray as xr
import time
import dask

In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,processes=28,name='pangeo',walltime='00:30:00',
                       job_extra=['--partition=cpu_p1','--account=egi@cpu'],memory='150GB',
                       interface='ib0') 
c= Client(cluster)
c



Client Scheduler: tcp://10.148.0.23:36347 Dashboard: http://10.148.0.23:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=28
#SBATCH --mem=140G
#SBATCH -t 00:30:00
#SBATCH --partition=cpu_p1
#SBATCH --account=egi@cpu

/gpfsscratch/rech/egi/rote001/miniconda2/envs/perf-pangeo/bin/python -m distributed.cli.dask_worker tcp://10.148.0.23:36347 --nthreads 1 --nprocs 28 --memory-limit 5.36GB --name name --nanny --death-timeout 60 --interface ib0



In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


In [ ]:
%time ds=xr.open_zarr('/gpfsscratch/rech/egi/rote001/eNATL60-BLBT02-SSH-1h/eNATL60-BLBT02-SSH-1h')

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
%time mean.load()